In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from utils.preprocessing import TextPurifier
from dataset import get_sms_dataset

In [2]:
data_df = get_sms_dataset(noStopwords=False, overwrite=True)

## 词频初步统计

In [9]:
from collections import Counter
def viewFrequency(df, column, cond):
    words = Counter()
    _words = df[df.loc[:, column]==cond].message.apply(lambda message: [word for word in message.split()])
    for word in _words:
        words.update(word)
    print(f'{column:s}-{cond}: ', words.most_common(50))


viewFrequency(data_df, 'target', 0)
viewFrequency(data_df, 'target', 1)

target-0:  [('i', 2940), ('you', 1943), ('to', 1554), ('the', 1122), ('a', 1056), ('u', 1018), ('$number', 922), ('and', 857), ('in', 818), ('me', 772), ('my', 750), ('is', 732), ('it', 711), ('that', 551), ('of', 525), ('for', 501), ('s', 478), ('have', 440), ('can', 439), ('so', 435), ('but', 434), ('your', 417), ('not', 415), ('are', 414), ('m', 405), ('on', 393), ('do', 384), ('at', 378), ('t', 375), ('if', 354), ('we', 354), ('will', 341), ('be', 335), ('gt', 318), ('lt', 316), ('get', 305), ('how', 304), ('now', 300), ('no', 298), ('just', 293), ('ok', 287), ('when', 287), ('up', 285), ('what', 273), ('with', 272), ('ll', 265), ('this', 252), ('go', 249), ('all', 245), ('ur', 241)]
target-1:  [('$number', 1661), ('to', 688), ('$phonenumber', 431), ('a', 377), ('call', 355), ('å', 299), ('you', 297), ('your', 264), ('free', 224), ('the', 206), ('for', 203), ('now', 202), ('or', 188), ('u', 174), ('txt', 163), ('is', 158), ('on', 144), ('ur', 144), ('have', 135), ('from', 131), ('m

如上所呈现的，Ham同Spam的top50词频存在蛮大的重叠，这些重叠很大程度上是由这些'stopword'引起的。


TODO: 同图展现词语出现率比对

因此选择去掉stopword仍然是有必要的

此处选择nltk.corp.stopwords词库来协助，并得到结果如下

In [10]:
data_df = get_sms_dataset(noStopwords=True, overwrite=True)
viewFrequency(data_df, 'target', 0)
viewFrequency(data_df, 'target', 1)

target-0:  [('u', 1018), ('$number', 922), ('gt', 318), ('lt', 316), ('get', 305), ('ok', 287), ('go', 249), ('ur', 241), ('know', 236), ('call', 236), ('good', 233), ('got', 232), ('like', 232), ('come', 227), ('day', 209), ('time', 201), ('love', 199), ('going', 169), ('one', 167), ('home', 165), ('want', 164), ('lor', 162), ('need', 158), ('sorry', 157), ('da', 150), ('still', 150), ('k', 145), ('n', 142), ('today', 139), ('see', 139), ('later', 135), ('dont', 132), ('r', 132), ('back', 130), ('send', 129), ('think', 128), ('take', 125), ('ì', 124), ('pls', 123), ('tell', 122), ('hi', 122), ('much', 115), ('oh', 113), ('well', 113), ('hope', 109), ('night', 109), ('hey', 107), ('wat', 106), ('happy', 106), ('dear', 106)]
target-1:  [('$number', 1661), ('$phonenumber', 431), ('call', 355), ('å', 299), ('free', 224), ('u', 174), ('txt', 163), ('ur', 144), ('mobile', 127), ('text', 125), ('stop', 121), ('claim', 113), ('reply', 104), ('www', 98), ('prize', 93), ('get', 86), ('cash', 76

In [12]:
X, y = data_df['message'], data_df['target']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
cv=CountVectorizer()
lr=LogisticRegression(max_iter=10000)
X_train=cv.fit_transform(X_train)

In [17]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(X_train,y_train)
pred_2=nb.predict(cv.transform(X_test))
score_2=accuracy_score(y_test,pred_2)
score_2

0.9928251121076234

In [23]:
from sklearn import metrics
metrics.confusion_matrix(y_test, pred_2)

array([[947,   2],
       [  6, 160]])

In [27]:
cv.transform(X_test)

<1115x6993 sparse matrix of type '<class 'numpy.int64'>'
	with 8616 stored elements in Compressed Sparse Row format>

In [53]:
y_pred_prob = nb.predict_proba(cv.transform(X_test))[:, 1]

In [54]:
metrics.roc_auc_score(y_test, y_pred_prob)

0.993823555549913